In [37]:
import pandas as pd
import warnings

warnings.simplefilter("ignore")
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)


- **df_label_fix: dataset yang telah dimodifikasi jumlah Augmented_Text pada masing - masing kategori sentimennya**
- **Jumlah: 6100 Positif & 3900 Negatif**
- **Ket: 1 Positif & 0 Negatif**


In [38]:
df_label_fix = pd.read_csv("../dataset/INA_Augmented_TextsPPKM_Labeled_PosNeg.csv", sep="\t")
df_label_fix.columns = df_label_fix.columns.str.lower()
df_label_fix.sample(5)


,date,user,tweet,sentiment
4271,2021-07-13 14:59:04+00:00,rklzrd,#mantap #ppkm #6minggu #mati https://t.co/UBTk...,1
5354,2021-07-05 00:05:38+00:00,cintahrumah,Pak Presiden Kapan Mundur Indonesia Darurat #P...,0
5256,2021-07-05 07:58:07+00:00,putridijee,Pernah Perhatian Kemudian Menghilang. #PPKM,0
4191,2021-07-14 04:28:05+00:00,Moca_Moonblues,PPKM = Pernah Perhatian Kemudian Menghilang \n...,0
5183,2021-07-05 13:47:04+00:00,betocation1,Minuman ini udah Rare neeh..... last stok Taji...,0


In [39]:
df_label_fix['sentiment'].value_counts()

0    3980
1    1958
Name: sentiment, dtype: int64

- **df_label_raw    : dataset yang masih murni hasil dari labeling yang belum mengalami modifikasi pada masing - masing kategori sentimennya**

In [40]:
df_label_raw = pd.read_csv("../dataset/INA_Augmented_TextsPPKM_Labeled_Pure.csv", sep="\t")
df_label_raw.columns = df_label_raw.columns.str.lower()
df_label_raw.sample(10)


,date,user,tweet,sentiment
17272,2021-07-10 22:26:59+00:00,lanangedan,Sebuah meaningful yang insightful of life dari...,1
14592,2021-07-19 07:55:10+00:00,alineadotid,"Juru Bicara Vaksinasi Kemenkes, Siti Nadia Tar...",1
4059,2021-09-24 13:22:21+00:00,tribunpontianak,Kapan Covid Berakhir\n#berita #BeritaTerkini #...,1
20022,2021-07-02 13:00:02+00:00,OfficialiNewsTV,Aksi dugaan pungutan liar yang dilakukan seora...,1
22674,2021-01-30 12:17:10+00:00,abebram82,Apa hasil #ppkm atau #psbb selama ini? Koq bis...,2
3094,2021-10-24 11:30:37+00:00,id_indicator,"Dibukanya kembali mal dan tempat wisata, menja...",1
20651,2021-06-23 00:39:08+00:00,binsarroy,"Covid mengganas, saatnya tegas, jangan sampai ...",2
3480,2021-10-12 10:04:18+00:00,mnctrijaya,#mnctrijaya #trijaya #trijayafm #trijayaupdate...,1
3511,2021-10-11 05:12:26+00:00,KangMuslihat8,PENYEKATAN JALUR 3 PILAR KOTA PASURUAN\nPenyek...,1
18330,2021-07-07 04:21:15+00:00,KahigamaID,Meskipun pandemi COVID sudah berlangsung &gt; ...,0


In [41]:
df_label_raw['sentiment'].value_counts()

1    17706
2     3980
0     1958
Name: sentiment, dtype: int64

In [109]:
import pandas as pd
import nlpaug.augmenter.word as naw


# Membuat fungsi untuk melakukan augmentasi teks
def augment_text(text):
    augmenter = naw.ContextualWordEmbsAug(
        model_path="cahya/bert-base-indonesian-522M", action="insert"
    )
    augmented_text = augmenter.augment(text)
    return augmented_text


# Membuat DataFrame contoh
data = {
    "Text": [
        "saya tidak suka dengan kebijakan ini",
        "saya senang melihat semangat presiden jokowi dalam bekerja",
    ]
}
df = pd.DataFrame(data)

# Mengaplikasikan augmentasi teks pada DataFrame
df["Augmented_Text"] = df["Text"].apply(augment_text)

# Menampilkan hasil
df


,Text,Augmented_Text
0,saya tidak suka dengan kebijakan ini,[jika saya tidak suka dengan kebijakan semacam...
1,saya senang melihat semangat presiden jokowi d...,[bisa saya senang melihat betapa semangat pres...


In [110]:
df['Augmented_Text'] = df['Augmented_Text'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Text            2 non-null      object
 1   Augmented_Text  2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [111]:
df["Augmented_Text"] = df["Augmented_Text"].str.replace('[','').str.replace(']','')
df["Augmented_Text"] = df["Augmented_Text"].str.replace("'",'')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Text            2 non-null      object
 1   Augmented_Text  2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [112]:
df.head()

,Text,Augmented_Text
0,saya tidak suka dengan kebijakan ini,jika saya tidak suka dengan kebijakan semacam ini
1,saya senang melihat semangat presiden jokowi d...,bisa saya senang melihat betapa semangat presi...


In [113]:
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Mengunduh resource yang diperlukan oleh NLTK (Natural Language Toolkit)
nltk.download("punkt")


[nltk_data] Downloading package punkt to C:\Users\SNOW-
[nltk_data]     WOLF\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [114]:
def remove_special_char(text):
    # Menghapus karakter khusus seperti tab, baris baru, dan backslash
    text = (
        text.replace("\\t", " ")
        .replace("\\n", " ")
        .replace("\\u", " ")
        .replace("\\", "")
    )
    # Menghapus karakter non-ASCII seperti emotikon
    text = text.encode("ASCII", "replace").decode("ASCII")
    # Menghapus mention, link, dan hashtag
    text = " ".join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())
    # Menghapus URL yang tidak lengkap
    return text.replace("http://", " ").replace("https://", " ")


def remove_number(text):
    # Menghapus angka dari teks
    return re.sub(r"\d+", "", text)


def remove_punctuation(text):
    # Menghapus tanda baca dari teks
    return text.translate(str.maketrans("", "", string.punctuation))


def remove_whitespace(text):
    # Menghapus spasi di awal dan akhir teks
    return text.strip()


def remove_multiple_whitespace(text):
    # Menghapus multiple whitespace menjadi satu spasi
    return re.sub("\s+", " ", text)


def remove_single_char(text):
    # Menghapus kata yang terdiri dari satu karakter saja
    return re.sub(r"\b[a-zA-Z]\b", "", text)


def tokenize(text):
    # Melakukan tokenisasi kata pada teks menggunakan NLTK
    return word_tokenize(text)


In [115]:
# Mengaplikasikan fungsi remove_special_char pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_special_char)

# Mengaplikasikan fungsi remove_number pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_number)

# Mengaplikasikan fungsi remove_punctuation pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_punctuation)

# Mengaplikasikan fungsi remove_whitespace pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_whitespace)

# Mengaplikasikan fungsi remove_multiple_whitespace pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_multiple_whitespace)

# Mengaplikasikan fungsi remove_single_char pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_single_char)

# Mengaplikasikan fungsi tokenize pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(tokenize)

# Menampilkan lima baris pertama dari DataFrame setelah dilakukan preprocessing teks
df.head()


,Text,Augmented_Text
0,saya tidak suka dengan kebijakan ini,"[jika, saya, tidak, suka, dengan, kebijakan, s..."
1,saya senang melihat semangat presiden jokowi d...,"[bisa, saya, senang, melihat, betapa, semangat..."
